In [123]:
import sklearn
import sklearn.datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn.metrics as metrics
import sklearn.naive_bayes
from sklearn.model_selection import train_test_split

import pandas as pd
from importlib import reload
from textblob import TextBlob

import preprocessing

In [9]:
target_folder = 'Dataset'
train_data = pd.read_csv(target_folder + '/train.csv')
test_data = pd.read_csv(target_folder + '/test.csv')
sumbission_example = pd.read_csv(target_folder + '/sample_submission.csv')
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
5,0001ea8717f6de06,0.5,0.5,0.5,0.5,0.5,0.5
6,00024115d4cbde0f,0.5,0.5,0.5,0.5,0.5,0.5
7,000247e83dcc1211,0.5,0.5,0.5,0.5,0.5,0.5
8,00025358d4737918,0.5,0.5,0.5,0.5,0.5,0.5
9,00026d1092fe71cc,0.5,0.5,0.5,0.5,0.5,0.5


In [124]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
X = vectorizer.fit_transform(train_data['comment_text'])
#X_test = vectorizer.transform(test_data['comment_text'])

In [125]:
yt = train_data['toxic']
yst = train_data['severe_toxic']
yo = train_data['obscene']
yth = train_data['threat']
yi = train_data['insult']
yih = train_data['identity_hate']

In [28]:
train_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [126]:
X_tr, X_te, yt_tr, yt_te, yst_tr, yst_te, yo_tr, yo_te, yth_tr, yth_te, yi_tr, yi_te, yih_tr, yih_te = train_test_split(X, yt, yst, yo, yth, yi, yih, test_size=0.33, random_state=42)

In [169]:
#clf = sklearn.linear_model.LogisticRegression(n_jobs=-1, verbose=2, solver='lbfgs', max_iter=15000)
clft = sklearn.naive_bayes.MultinomialNB(alpha=.6)
clft.fit(X_tr, yt_tr)
predt = clft.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yt_te, predt))

clfst = sklearn.naive_bayes.MultinomialNB(alpha=.2)
clfst.fit(X_tr, yst_tr)
predst = clfst.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yst_te, predst))

clfo = sklearn.naive_bayes.MultinomialNB(alpha=.5)
clfo.fit(X_tr, yo_tr)
predo = clfo.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yo_te, predo))

clfth = sklearn.naive_bayes.MultinomialNB(alpha=.07)
clfth.fit(X_tr, yth_tr)
predth = clfth.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yth_te, predth))

clfi = sklearn.naive_bayes.MultinomialNB(alpha=.5)
clfi.fit(X_tr, yi_tr)
predi = clfi.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yi_te, predi))

clfih = sklearn.naive_bayes.MultinomialNB(alpha=.2)
clfih.fit(X_tr, yih_tr)
predih = clfih.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yih_te, predih))

0.9542636086598915
0.9758711431182266
0.963111017832109
0.9708151695062212
0.960371587637548
0.9614099531060089


In [170]:
X_test = vectorizer.transform(test_data['comment_text'])

In [171]:
predt = clft.predict_proba(X_test)[:,1]
predst = clfst.predict_proba(X_test)[:,1]
predo = clfo.predict_proba(X_test)[:,1]
predth = clfth.predict_proba(X_test)[:,1]
predi = clfi.predict_proba(X_test)[:,1]
predih = clfih.predict_proba(X_test)[:,1]

In [172]:
sumbission_example['toxic']=predt
sumbission_example['severe_toxic']=predst
sumbission_example['obscene']=predo
sumbission_example['threat']=predth
sumbission_example['insult']=predi
sumbission_example['identity_hate']=predih

In [173]:
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997132,0.656453,0.991428,0.120429,0.974775,0.703437
1,0000247867823ef7,0.015078,0.000434,0.007394,0.000156,0.004286,0.000758
2,00013b17ad220c46,0.026269,0.000533,0.011008,0.000075,0.010103,0.001310
3,00017563c3f7919a,0.005898,0.000145,0.002135,0.000023,0.002167,0.000099
4,00017695ad8997eb,0.040821,0.003894,0.025155,0.001359,0.023148,0.002919
5,0001ea8717f6de06,0.014653,0.000263,0.004768,0.000080,0.005711,0.000296
6,00024115d4cbde0f,0.006330,0.000098,0.002814,0.000009,0.001967,0.000086
7,000247e83dcc1211,0.379985,0.010608,0.159356,0.011652,0.168955,0.011949
8,00025358d4737918,0.019457,0.000209,0.009390,0.000159,0.005106,0.000380
9,00026d1092fe71cc,0.005353,0.000125,0.003329,0.000076,0.002978,0.000239


In [174]:
sumbission_example.to_csv('nb_tfidf0.csv', index=False)